# Demonstration Task 4.3: Tool integration for calibration & UQ 
We demonstrate a three-step calibration and uncertainty prediction framework as a prototype workflow:
1. Sampling : Generate a mapping of a chosen error metric using a set of chosen model parameters. Return a netcdf of this mapping. 
2. Sensitivity : Learn the errors using PCE. Return a netcdf of Sobol indices : dependency on physical parameters, and aggregation.
3. Calibration + UQ : Calibrate the most senstive parameters and return their posterior distribution.

Integrated tools: WindIO-based WIFA + PCE + umbra

<font color='blue'>General libraries</font> 

In [ ]:
%matplotlib inline
import xarray as xr
import matplotlib.pyplot as plt

from windIO.utils.yml_utils import validate_yaml, load_yaml
from windIO.utils import plant_schemas_path

<font color='blue'> WIFA libraries</font> 

In [ ]:
from generalize_database_creation import create_error_database
from perform_PCE_on_database import *

<font color='blue'>Toy problem data </font> 

Flow field based

In [ ]:
calibration_windio_input_yaml = "1WT_simulations/windIO_1WT/wind_energy_system/system.yaml"
# validate_yaml(calibration_windio_input_yaml, plant_schemas_path + "wind_energy_system.yaml")
calibration_windio_input_data = load_yaml(calibration_windio_input_yaml)
#
calibration_reference_flow_field = xr.load_dataset('1WT_simulations/result_code_saturne_1WT_FULL/flow_field_interpolated.nc')
calibration_reference_turbine_data = xr.load_dataset('1WT_simulations/result_code_saturne_1WT_FULL/turbine_data.nc')
calibration_reference_physical_inputs= xr.load_dataset('1WT_simulations/windIO_1WT/plant_energy_resource/1WT_calibration_data_IEA15MW.nc')

Turbine power based

In [ ]:
calibration_windio_input_yaml = "farm_simulations/windio_Farm_ABL_IEA15/wind_energy_system/system_staggered_DX5D_DY5D_Turbine_NumberNumber25.yaml"
calibration_windio_input_data = load_yaml(calibration_windio_input_yaml)

flow_field_file = None
turbine_data_file = 'farm_simulations/Result_code_saturne_Farm_calibration_data_LIGHT/farm_result/turbine_data_staggered_DX5D_DY5D_Turbine_NumberNumber25.nc'
inputs_file = 'farm_simulations/windio_Farm_ABL_IEA15/plant_energy_resource/Farm_calibration_data_IEA15MW.nc'

calibration_reference_flow_field = None
calibration_reference_turbine_data = xr.load_dataset(turbine_data_file)
calibration_reference_physical_inputs= xr.load_dataset(inputs_file)

<font color='blue'> Step 1 : error mapping </font> 

##### Run the sampling and store into netcdf

In [ ]:
if(True): #set true to generate the netcdf
    calibration_model_param_config = {
        "attributes.analysis.wind_deficit_model.wake_expansion_coefficient.k_b": (0.01, 0.3),
        "attributes.analysis.blockage_model.ss_alpha": (0.01, 0.3)
    }
    reference_qoi = {'power': turbine_data_file} # , 'wind_speed': flow_field_file
    UQ_mapping_results = create_error_database(calibration_windio_input_data, calibration_model_param_config, \
                                  calibration_reference_physical_inputs, reference_qoi, \
                                  n_samples=50, seed=3)
    UQ_mapping_results.to_netcdf('UQ_mapping_results.nc')

##### Read a precomputed sampling from netcdf

In [ ]:
UQ_mapping_results = xr.load_dataset('UQ_mapping_results.nc')
UQ_mapping_results_keys = list(UQ_mapping_results.keys())
UQ_mapping_results_coords = list(UQ_mapping_results.coords)

In [ ]:
output_variable_name = "power_diff" # "wind_diff"

##### Illustrate content

In [ ]:
#
print("Physical fields and parameters :",UQ_mapping_results_keys)
print("Coordinates including the model parameters :",UQ_mapping_results_coords)
#
fig, ax = plt.subplots(1,1,figsize=(4,2))
plt.scatter(UQ_mapping_results.ss_alpha, UQ_mapping_results.k_b, c=UQ_mapping_results[output_variable_name].min('time'))
cbar = plt.colorbar()
plt.xlabel('ss_alpha')
plt.ylabel('k')
cbar.set_label('Error')
plt.show()

<font color='blue'> Step 2 : PCE learning - v1 : given time </font>

##### Physical and model parameters
<font color='red'>TODO : automatically decide what physical_inputs and error_metrics for the netcdf</font> 

In [ ]:
physical_inputs_varnames = ['z0', 'ABL_height', 'wind_speed']
model_param_varnames = [coord for coord in UQ_mapping_results_coords if (not(coord in ['time','z','sample']))]
print("Physical parameters : ", physical_inputs_varnames)
print("Model parameters : ", model_param_varnames)
print("Error metric name : ", output_variable_name)

##### Few prints to illustrate the content

In [ ]:
sample_size=len(UQ_mapping_results.sample)
physical_size = len(UQ_mapping_results.time)
print("\nSample size is : ",sample_size, "(realizations of model params).", \
      " Number of physical times is : ", physical_size, "(realizations of physical params).", \
      f"\nResulting shape of the error_metric table ({output_variable_name}) is :",np.array(UQ_mapping_results.isel().get(output_variable_name)).shape)

In [ ]:
it=0 ; isp=0
print("\nVerification prints for time ",UQ_mapping_results.time.data[it], "and sample id ",isp, \
      ":\n  Physical params: ", physical_inputs_varnames, "=",[str(UQ_mapping_results.isel(time=it, sample=isp).get(physical_inputs_varnames[i]).data) \
                                                             for i in range(len(physical_inputs_varnames))], \
      "\n  Model params: ", model_param_varnames, "=",[str(UQ_mapping_results.isel(time=it, sample=isp).get(model_param_varnames[i]).data) \
                                                             for i in range(len(model_param_varnames))], \
      "\n  Error metric (relative to reference): ",output_variable_name," = ",  UQ_mapping_results.isel(time=it, sample=isp)[output_variable_name].data
)

##### Learn PCE and get Sobol Indices

In [ ]:
### PCE example for given time it
it=10 #selected evaluation time
PCE_deg=5
marginals = ["kernel", "kernel"] #agnostic=kernel. choices: uniform, gaussian
copula = "independent"  #dependency btw the random variables, choices: gaussian, independent
Sobol_indices_1stOrder, Sobol_indices_Total, PCE_metamodel  =  build_PCE_for_given_time(UQ_mapping_results, model_param_varnames, \
                                                                                        output_variable_name,it,PCE_deg, marginals, copula)

print("Sobol indices at time id, ",it,"and model params ", model_param_varnames," : ",Sobol_indices_1stOrder)
example_k=0.1 ; example_alpha_ss = 0.05
print("Example of metamodel evaluation :", PCE_metamodel(np.array([example_k,example_alpha_ss]))[0]) #error(k, alpha_ss) ~ PCE(k, alpha_ss)

In [ ]:
### For all times
allTimes_Sobol_indices_1stOrder, allTimes_Sobol_indices_Total, \
    allTimes_PCE_metamodels  = build_PCE_for_all_times(UQ_mapping_results, model_param_varnames, \
                                                       output_variable_name,PCE_deg, marginals, copula)

In [ ]:
#Print chosen time
it=20
print("Sobol indices at time id, ",it,"and model params ", model_param_varnames," : ",allTimes_Sobol_indices_1stOrder[it,:])
example_k=0.1 ; example_alpha_ss = 0.05
print("Example of metamodel evaluation :", allTimes_PCE_metamodels[it](np.array([example_k,example_alpha_ss]))[0]) #error(k, alpha_ss) ~ PCE(k, alpha_ss)

##### Sobol indices plot

In [ ]:
#########Plot Sobol indices############
it=20 #chosen time
fig,ax = plt.subplots(1,1,figsize=(4,3))
##### Bar positions
bar_width=0.24
bar_positions1 = np.arange(len(model_param_varnames)) + 0.5*bar_width
bar_positions2 = np.arange(len(model_param_varnames)) + 1.5*bar_width
values1 = []
values2 = []
for j in range(len(model_param_varnames)):
    values1.append(allTimes_Sobol_indices_1stOrder[it,j])
    values2.append(allTimes_Sobol_indices_Total[it,j])
#####
# Create bar plots
ax.bar(bar_positions1, values1, width=bar_width, label='$1^{st}$ Order', color="blue")
ax.bar(bar_positions2, values2, width=bar_width, label='Total', color="green")

# Set labels
ax.set_ylabel("Sobols, time "+str(it))
ax.set_xticks(bar_positions2) , ax.set_xticklabels(model_param_varnames)
ax.legend(loc="upper right", fontsize=12)
ax.tick_params(axis='both', which='major', labelsize=15)
ax.xaxis.label.set_size(15) , ax.yaxis.label.set_size(15)
ax.set_ylim(0.0,1.1)

# Show the plot
plt.tight_layout()
plt.show()

<font color='blue'> Step 2 : PCE learning - v2 : aggregated </font> 

In [ ]:
### PCE for selected modes
PCE_deg=5
marginals = ["kernel", "kernel"] #agnostic=kernel. choices: uniform, gaussian
copula = "independent"  #dependency btw the random variables, choices: gaussian, independent

#########Set the data from previous###############
aggregated_Sobol_indices_1stOrder, aggregated_Sobol_indices_Total  =  build_POD_PCE(UQ_mapping_results, model_param_varnames, \
                                                                                        output_variable_name,PCE_deg, marginals,copula,target_POD_EVR=0.99)

print("Aggregated Sobol indices for model params ", model_param_varnames," : ",aggregated_Sobol_indices_1stOrder)

In [ ]:
#########Plot Sobol indices############
fig,ax = plt.subplots(1,1,figsize=(4,3))
##### Bar positions
bar_width=0.24
bar_positions1 = np.arange(len(model_param_varnames)) + 0.5*bar_width
bar_positions2 = np.arange(len(model_param_varnames)) + 1.5*bar_width
values1 = []
values2 = []
for j in range(len(model_param_varnames)):
    values1.append(aggregated_Sobol_indices_1stOrder[j])
    values2.append(aggregated_Sobol_indices_Total[j])
#####
# Create bar plots
ax.bar(bar_positions1, values1, width=bar_width, label='$1^{st}$ Order', color="blue")
ax.bar(bar_positions2, values2, width=bar_width, label='Total', color="green")

# Set labels
ax.set_ylabel(" Sobols, aggregated")
ax.set_xticks(bar_positions2) , ax.set_xticklabels(model_param_varnames)
ax.legend(loc="upper right", fontsize=12)
ax.tick_params(axis='both', which='major', labelsize=15)
ax.xaxis.label.set_size(15) , ax.yaxis.label.set_size(15)
ax.set_ylim(0.0,1.1)

# Show the plot
plt.tight_layout()
plt.show()

#### Storing Sensitivity Analysis into netcdf file

In [ ]:
# Add parameter values to dataset
import xarray as xr
SA_data = xr.Dataset(
    data_vars=dict(Sobols_1stOrder=(["time","model_params"], allTimes_Sobol_indices_1stOrder), 
                   Sobols_Total=(["time","model_params"], allTimes_Sobol_indices_Total), 
                   Aggregated_Sobols_1stOrder=(["model_params"], aggregated_Sobol_indices_1stOrder),
                   Aggregated_Sobols_Total=(["model_params"], aggregated_Sobol_indices_Total)
                  ),
    coords={
                'time': xr.DataArray(UQ_mapping_results.time,dims=['time'],coords={'time': UQ_mapping_results.time}),                 
                'model_params': xr.DataArray(model_param_varnames, dims=['model_params'], coords={'model_params': model_param_varnames})            
        }
    )

# Add reference data
refdat = UQ_mapping_results
for var in refdat.data_vars:
    SA_data[var] = refdat[var]    

print(SA_data)
SA_data.to_netcdf('SA_results.nc')

<font color='blue'> Step 3 : Bayesian calibration with UQ </font>

Based on the Sobol indices, we select the wake expansion rate as the most sensitive parameter. This posterior distribution of this parameter is now approximated with samples via Bayesian UQ. 

In [ ]:
import umbra
import ipyparallel as ipp

Load the wind-farm flow model via WIFA based on the WindIO yaml:

In [ ]:
fm = umbra.flow_model.WIFAModel(calibration_windio_input_yaml)
fm.name_model

Normalize the data with the upstream power:

In [ ]:
data = np.diag(1/fm.p_infty()) @ calibration_reference_turbine_data.power.to_numpy().T

Define the posterior of the parameters via their prior and the likelihood of the data given the model and parameters

In [ ]:
# Prior
prior = umbra.bayesian_model.Prior(['k_b'], [umbra.distributions.Uniform(0, 1)])
for i in range(umbra.bayesian_model.get_nb_bias_categories(fm)):
    prior.add(f'sigma_B{i}', umbra.distributions.Exponential(0.1))

# Likelihood    
likelihood = umbra.bayesian_model.LikelihoodTurbinePower(flow_model=fm, data=data)
# Bayesian model (posterior)
model = umbra.bayesian_model.BayesianModel(prior, likelihood, name='umbra4wifa-demo')

Sample from the posterior distribution with a parallelized MCMC algorithm

In [ ]:
# Select a sampling algorithm
sampler = umbra.sampler.TMCMC(model, parallel=True, n_samples=160)
# Sample in parallel
with ipp.Cluster(engines='mpi', n=8) as rc:
    view = rc.broadcast_view()
    r = view.apply_sync(sampler.sample)
inference_data = r[0]
# Retrieve the samples
samples = inference_data.samples

Visualize the posterior distribution of the wake expansion rate

In [ ]:
plt.hist(samples[:,0], color='k'); plt.xlim([0.02,0.1])
plt.xlabel('$k_b$'); plt.ylabel('N (posterior samples)'); plt.show()

Propagate the quantified uncertainty through the model and the likelihood of the data to construct the posterior predictive distribution

In [ ]:
post_pred = model.posterior_predictive(samples)
umbra.plotting.PPC(np.arange(fm.nb_turbines)+1, data, [(post_pred, 'Post Pred')], xlabel='Turbine', ylabel='$P/P_\infty$ [-]')

Since the data seems plausible under the posterior predictive distribution, the UQ is valid.